In [46]:
import pandas as pd 
import os

In [47]:
folder_path= './PassengerVehicle_Stats'
dfs=[]
files= [f for f in os.listdir(folder_path) if f.endswith('.csv')]

for file in files:
    file_path = os.path.join(folder_path,file)
    df1 = pd.read_csv(file_path)
    dfs.append(df1)

combine_df = pd.concat(dfs, ignore_index=True)
combine_df.to_csv('./vehicles_df.csv', index= False)

In [48]:
vehicles_df = pd.read_csv('vehicles_df.csv')
vehicles_df.isna().sum()

Unnamed: 0                                0
Public Vehicle Number                     0
Status                                    0
Vehicle Make                           7668
Vehicle Model                          7852
Vehicle Model Year                     7768
Vehicle Color                          7944
Vehicle Fuel Source                       0
Wheelchair Accessible                     0
Company Name                              0
Address                                7144
City                                   7144
State                                  7144
ZIP Code                               7144
Taxi Affiliation                      37016
Taxi Medallion License Management     37124
Record ID                                 0
dtype: int64

In [49]:
print(vehicles_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66408 entries, 0 to 66407
Data columns (total 17 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Unnamed: 0                          66408 non-null  int64  
 1   Public Vehicle Number               66408 non-null  int64  
 2   Status                              66408 non-null  object 
 3   Vehicle Make                        58740 non-null  object 
 4   Vehicle Model                       58556 non-null  object 
 5   Vehicle Model Year                  58640 non-null  float64
 6   Vehicle Color                       58464 non-null  object 
 7   Vehicle Fuel Source                 66408 non-null  object 
 8   Wheelchair Accessible               66408 non-null  object 
 9   Company Name                        66408 non-null  object 
 10  Address                             59264 non-null  object 
 11  City                                59264

In [50]:
print(vehicles_df.head())

   Unnamed: 0  Public Vehicle Number    Status Vehicle Make Vehicle Model  \
0        1286                  12009  RESERVED    CHEVROLET       EXPRESS   
1        2095                  12248  INACTIVE     MERCEDES      SPRINTER   
2        7950                  13527  INACTIVE     VAN HOOL         TD925   
3        8700                  12248  INACTIVE     MERCEDES      SPRINTER   
4        9359                  13528  INACTIVE     VAN HOOL         TD925   

   Vehicle Model Year Vehicle Color Vehicle Fuel Source Wheelchair Accessible  \
0              2014.0         BLACK          Bio-Diesel                     N   
1              2010.0        SILVER          Bio-Diesel                     N   
2              2008.0           RED          Bio-Diesel                     N   
3              2010.0        SILVER          Bio-Diesel                     N   
4              2008.0           RED          Bio-Diesel                     N   

                              Company Name        

To remove the duplicates.

In [51]:
vehicles_df_cleaned = vehicles_df.drop_duplicates(subset='Public Vehicle Number', keep='first')
vehicles_df_cleaned.shape

(14473, 17)

In [52]:
vehicles_df_cleaned.isnull().sum()

Unnamed: 0                               0
Public Vehicle Number                    0
Status                                   0
Vehicle Make                          1801
Vehicle Model                         1837
Vehicle Model Year                    1819
Vehicle Color                         1855
Vehicle Fuel Source                      0
Wheelchair Accessible                    0
Company Name                             0
Address                               1688
City                                  1688
State                                 1688
ZIP Code                              1688
Taxi Affiliation                      8091
Taxi Medallion License Management     8114
Record ID                                0
dtype: int64

In [55]:
columns_to_check = ['Vehicle Make', 'Vehicle Model', 'Vehicle Model Year',
                     'Vehicle Color', 'Address', 'City', 'State', 'ZIP Code']

vehicles_df_cleaned.dropna(subset=columns_to_check, how = 'all', inplace=True)
vehicles_df_cleaned.shape

C:\Users\DELL\AppData\Local\Temp\ipykernel_1064\294867357.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicles_df_cleaned.dropna(subset=columns_to_check, how = 'all', inplace=True)


(12785, 17)

In [56]:
vehicles_df_cleaned.isnull().sum()

Unnamed: 0                               0
Public Vehicle Number                    0
Status                                   0
Vehicle Make                           113
Vehicle Model                          149
Vehicle Model Year                     131
Vehicle Color                          167
Vehicle Fuel Source                      0
Wheelchair Accessible                    0
Company Name                             0
Address                                  0
City                                     0
State                                    0
ZIP Code                                 0
Taxi Affiliation                      6403
Taxi Medallion License Management     6426
Record ID                                0
dtype: int64